In [ ]:
import torch, platform, os
print("Python", platform.python_version())
print("CUDA available:", torch.cuda.is_available(), "| device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu")

Python 3.11.13
CUDA available: True | device: Tesla T4


In [ ]:
!git clone --depth 1 https://github.com/KEggensperger/DistNet
!git clone --depth 1 https://github.com/PriorLabs/TabPFN

Cloning into 'DistNet'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 25 (delta 1), reused 25 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 27.79 KiB | 836.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.
Cloning into 'TabPFN'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 105 (delta 10), reused 63 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (105/105), 1.48 MiB | 5.27 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
%pip install -q --force-reinstall "torch==2.2.2" "torchvision==0.17.2" "torchaudio==2.2.2"
%pip install -q --force-reinstall "numpy==1.26.4" "ConfigSpace==1.2.1"
%pip install -q tabpfn tabpfn-extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19

In [ ]:
import numpy, torch, tabpfn
print("NumPy :", numpy.__version__)   # should be 1.26.4
print("Torch :", torch.__version__)   # should be 2.2.2
print("TabPFN:", tabpfn.__version__)  # 2.1.0 (or newer if you upgraded)

NumPy : 1.26.4
Torch : 2.2.2+cu121
TabPFN: 2.1.1


In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path("DistNet").resolve()))
print("Paths ready.")

Paths ready.


In [ ]:
%%bash
python - <<'PY'
import sys, pathlib
sys.path.append(str(pathlib.Path("DistNet").resolve()))
print("Paths ready.")
PY

Paths ready.


In [ ]:
%%bash
cd DistNet
curl -L -o DistNetData.zip "http://www.ml4aad.org/wp-content/uploads/2018/04/DistNetData.zip"
unzip -q DistNetData.zip -d data
mv data/DistNetData/* data/
rm -rf data/DistNetData DistNetData.zip
cd ..

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  181M  100  181M    0     0  10.3M      0  0:00:17  0:00:17 --:--:-- 11.6M


In [ ]:
# ⬇️ Writes DistNet/scripts/eval_tabpfn_iid.py to report NLL in Standardized (Z-score) space
from textwrap import dedent
from pathlib import Path

code = dedent(r'''
#!/usr/bin/env python3
import argparse, os, pickle, sys, pathlib
import numpy as np
import torch
from tabpfn import TabPFNRegressor

# Make DistNet modules importable regardless of CWD
ROOT = pathlib.Path(__file__).resolve().parent.parent
sys.path.insert(0, str(ROOT))

from helper import load_data, preprocess, data_source_release
from sklearn.model_selection import KFold


def predict_nll(reg,
                X_vl_flat32,
                Y_vl_flat32_transformed,
                batch_size: int = 4096):
    """
    Compute continuous NLL in batches on the transformed (standardized) data.
    """
    n = X_vl_flat32.shape[0]
    out_chunks, pos = [], 0
    eps = 1e-12

    while pos < n:
        end = min(pos + batch_size, n)
        with torch.no_grad():
            full = reg.predict(X_vl_flat32[pos:end], output_type="full")

            criterion = full.get("criterion", None) or getattr(reg, "renormalized_criterion_", None) or getattr(reg, "bardist_", None)
            if criterion is None:
                raise RuntimeError("TabPFN criterion not found. Update tabpfen or keep output_type='full'.")

            dev = criterion.borders.device
            logits = torch.as_tensor(full["logits"], device=dev, dtype=torch.float32)
            y_true_transformed = torch.as_tensor(Y_vl_flat32_transformed[pos:end], device=dev, dtype=torch.float32).view(-1)

            # NLL in transformed (Z-score) space
            pdf_transformed = criterion.pdf(logits, y_true_transformed).clamp_min(eps)
            nll = -torch.log(pdf_transformed)

        out_chunks.append(nll.detach().cpu())
        pos = end

    return torch.cat(out_chunks, dim=0).numpy()


def main():
    p = argparse.ArgumentParser()
    sc_dict = data_source_release.get_sc_dict()
    p.add_argument("--scenario", required=True, choices=sc_dict.keys())
    p.add_argument("--fold", type=int, required=True)
    p.add_argument("--save", required=True)

    p.add_argument("--subsample_mode", choices=["first", "flattened_random", "instance_wise"], default="instance_wise",
                   help="Subsampling strategy. 'instance_wise' is recommended for better results.")
    p.add_argument("--cap_rows", type=int, default=4096,
                   help="Max training context rows. For instance_wise, this is used to calculate the number of instances to sample.")
    p.add_argument("--passes", type=int, default=2,
                   help="Repeat iid subsample + re-fit and average NLLs for more stable results.")
    p.add_argument("--val_batch", type=int, default=4096)
    p.add_argument("--seed", type=int, default=1)

    args = p.parse_args()
    assert 0 <= args.fold <= 9

    print(f">> TabPFN (Subsample: {args.subsample_mode}): Standardization (Z-score). NLL in STANDARDIZED space.")

    data_dir = data_source_release.get_data_dir()
    runtimes, features, _ = load_data.get_data(
        scenario=args.scenario, data_dir=data_dir,
        sc_dict=sc_dict, retrieve=sc_dict[args.scenario]["use"])

    X, Y = np.asarray(features), np.asarray(runtimes)

    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    tr_idx, vl_idx = list(kf.split(np.arange(X.shape[0])))[args.fold]

    X_tr_raw, X_vl_raw = X[tr_idx], X[vl_idx]
    Y_tr_raw, Y_vl_raw = Y[tr_idx], Y[vl_idx]

    X_tr, X_vl = preprocess.preprocess_features(X_tr_raw, X_vl_raw, scal="meanstd")
    n_reps = Y_tr_raw.shape[1]

    X_tr_flat = np.repeat(X_tr, n_reps, axis=0).astype(np.float32)
    Y_tr_flat = Y_tr_raw.flatten().astype(np.float32)
    X_vl_flat = np.repeat(X_vl, n_reps, axis=0).astype(np.float32)
    Y_vl_flat = Y_vl_raw.flatten().astype(np.float32)

    # Standardize targets using Mean and Std Dev from TRAINING data
    train_mean = np.mean(Y_tr_flat)
    train_std = np.std(Y_tr_flat)
    if train_std < 1e-8:
        train_std = 1.0

    Y_tr_flat_std = ((Y_tr_flat - train_mean) / train_std).astype(np.float32)
    Y_vl_flat_std = ((Y_vl_flat - train_mean) / train_std).astype(np.float32)

    print(f"  Train Mean: {train_mean:.4f}, Train Std Dev: {train_std:.4f}")

    nll_passes = []
    for pass_id in range(args.passes):
        rng_pass = np.random.RandomState(args.seed + pass_id)

        if args.subsample_mode == "instance_wise":
            n_instances_tr = X_tr.shape[0]
            n_instances_to_sample = max(1, min(n_instances_tr, args.cap_rows // n_reps))
            print(f"  [Pass {pass_id+1}/{args.passes}] Sampling {n_instances_to_sample}/{n_instances_tr} instances...")

            sub_instance_idx = rng_pass.choice(n_instances_tr, size=n_instances_to_sample, replace=True)

            X_sub_instances = X_tr[sub_instance_idx]
            Y_tr_std = Y_tr_flat_std.reshape(Y_tr_raw.shape)
            Y_sub_instances_std = Y_tr_std[sub_instance_idx]

            X_sub = np.repeat(X_sub_instances, n_reps, axis=0).astype(np.float32)
            Y_sub = Y_sub_instances_std.flatten().astype(np.float32)

        else: # flattened_random or first
            N_flat = X_tr_flat.shape[0]
            cap = min(args.cap_rows, N_flat)
            print(f"  [Pass {pass_id+1}/{args.passes}] Sampling {cap}/{N_flat} rows...")
            sub_idx = rng_pass.choice(N_flat, size=cap, replace=True)
            X_sub, Y_sub = X_tr_flat[sub_idx], Y_tr_flat_std[sub_idx]

        device = "cuda" if torch.cuda.is_available() else "cpu"
        reg = TabPFNRegressor(device=device, ignore_pretraining_limits=True)
        reg.fit(X_sub, Y_sub)

        nll_rep = predict_nll(reg, X_vl_flat, Y_vl_flat_std, batch_size=args.val_batch)
        val_pred = nll_rep.reshape(-1, n_reps).mean(axis=1)
        nll_passes.append(val_pred)

    val_pred_mean = np.mean(np.stack(nll_passes, axis=0), axis=0)

    os.makedirs(args.save, exist_ok=True)
    add_info = {
        "scenario": args.scenario, "fold": args.fold, "model": "tabpfn",
        "cap_rows": args.cap_rows, "passes": args.passes, "subsample_mode": args.subsample_mode,
        "seed": args.seed, "y_transform": "standardize", "report_space": "standardized_zscore_space",
        "scale_meta": {"transform": "zscore", "train_mean": train_mean, "train_std": train_std}
    }
    filename = (f"{args.scenario}.full.tabpfn.sample_{args.subsample_mode}"
                f".cap{args.cap_rows}.passes{args.passes}.fold{args.fold}.rep_standardized_space.pkl")
    fp = os.path.join(args.save, filename)

    with open(fp, "wb") as fh:
        pickle.dump([None, val_pred_mean, add_info], fh, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"Saved {fp} | Mean NLL: {float(np.mean(val_pred_mean)):.4f}")


if __name__ == "__main__":
    main()
''')

Path("DistNet/scripts").mkdir(parents=True, exist_ok=True)
Path("DistNet/scripts/eval_tabpfn_iid.py").write_text(code)
print("NLL is now reported in the Standardized (Z-score) space with NO correction.")

In [ ]:
# uninstall first to avoid mixups
%pip uninstall -y torch torchvision torchaudio

# install cu121 wheels for 2.5.1
%pip install -q --no-cache-dir --force-reinstall \
  --index-url https://download.pytorch.org/whl/cu121 \
  "torch==2.5.1" "torchvision==0.20.1" "torchaudio==2.5.1"

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 129.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 136.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 146.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 134.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 228.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 128.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 122.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install -q tabpfn tabpfn-extensions

In [ ]:
import torch
import tabpfn  # <-- This line was missing

print("Torch :", torch.__version__)
print("TabPFN:", tabpfn.__version__, "from", tabpfn.__file__)

# The check for 'enable_gqa' is removed.
# The tabpfen library automatically uses the optimized attention mechanism
# available in the installed PyTorch version (2.5.1), so this check is not needed.
print("\nPyTorch version 2.5.1 installed, which includes optimized attention backends for TabPFN.")

Torch : 2.5.1+cu121
TabPFN: 2.1.1 from /usr/local/lib/python3.11/dist-packages/tabpfn/__init__.py

PyTorch version 2.5.1 installed, which includes optimized attention backends for TabPFN.


In [ ]:
!python DistNet/scripts/eval_tabpfn_iid.py \
  --scenario clasp_factoring \
  --fold 0 \
  --save results_sanity_check \
  --cap_rows 2048 \
  --passes 1 \
  --subsample_mode instance_wise \
  --val_batch 4096 \
  --seed 1

In [ ]:
# 🔁 Run TabPFN (log1p target, original-space NLL, i.i.d. subsampling) across scenarios × 10 folds

SCENARIOS = [
    "clasp_factoring",
    "spear_qcp",
    "yalsat_qcp",
    "lpg-zeno",
]

SAVE_ROOT  = "results_iid"   # scenario subfolders will be created
CAP_ROWS   = 4096            # iid context size per fold (increase if VRAM allows)
PASSES     = 2               # independent iid subsamples to average
SUBSAMPLE  = "random"        # i.i.d. subsampling (with replacement handled inside the script)
VAL_BATCH  = 4096            # reduce if you hit OOM
SEED       = 1

import subprocess, os, sys, time

os.makedirs(SAVE_ROOT, exist_ok=True)
t_all = time.time()

for sc in SCENARIOS:
    save_dir = os.path.join(SAVE_ROOT, sc)
    os.makedirs(save_dir, exist_ok=True)
    print(f"\n=== Scenario: {sc} ===")
    t0 = time.time()

    for fold in range(10):
        print(f"→ fold {fold}")
        cmd = [
            sys.executable, "DistNet/scripts/eval_tabpfn_iid.py",
            "--scenario", sc,
            "--fold", str(fold),
            "--save", save_dir,
            "--cap_rows", str(CAP_ROWS),
            "--passes", str(PASSES),
            "--subsample_mode", SUBSAMPLE,   # default is 'random', kept explicit
            "--val_batch", str(VAL_BATCH),
            "--seed", str(SEED),
            # no --y_transform / --report_space flags needed (defaults handle it)
        ]
        try:
            subprocess.run(cmd, check=True)
        except subprocess.CalledProcessError as e:
            print(f"✗ fold {fold} failed (exit {e.returncode}); continuing…")

    print(f"✔ finished {sc} in {(time.time()-t0)/60:.1f} min")

print(f"\n✔ all scenarios done in {(time.time()-t_all)/3600:.2f} h")